In [1]:
import pandas as pd
import numpy as np
import import_ipynb
import sys
import math 
import seaborn as sea
import datetime as dt
import Fuzzy_Prediction_Beta as fz
from datetime import timedelta
from operator import itemgetter
from functools import partial
url = "in.csv"

importing Jupyter notebook from Fuzzy_Prediction_Beta.ipynb


In [2]:
dataset = pd.read_csv(url, error_bad_lines=False)

In [3]:
selected = dataset[['cod_location','date_id','total_cases']]

In [4]:
selected = selected.groupby('cod_location').tail(5)

In [5]:
selected.head()
#Input na rede neural
selected.to_csv('input.csv')

In [6]:
def aver(input):
    medias = input.groupby(['cod_location'])['total_cases'].mean()
    return medias

In [7]:
def diff(input):
    difference = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

In [8]:
def first(input):
    first = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.head(1).values[0])
    return first

In [9]:
def last(input):
    last = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0])
    return last

In [10]:
def fuzzifyErrorsPos(erro, media, maximus):
    return fz.trapezio(erro,[(media + ((media * 26) / 100)), (media + ((media * 40) / 100)), maximus, maximus])

In [11]:
def fuzzifyErrorPos(erro, media):
    return fz.triangulo(erro, [(media - (media + ((media * 8) / 100))), (media + ((media * 18) / 100)), (media + ((media * 28) / 100))])

In [12]:
def fuzzifyErrorZ(erro, media):
    return fz.triangulo(erro, [(media - ((media * 10) / 100)), media, (media + ((media * 10) / 100))])

In [13]:
def fuzzifyErrorNeg(erro, media):
    return fz.triangulo(erro, [(media - ((media * 28) / 100)), (media - ((media * 18) / 100)), (media - ((media * 8) / 100))])

In [14]:
def fuzzifyErrorsNeg(erro, media):
    return fz.trapezio(erro, [0, 0, (media - ((media * 40) / 100)), (media - ((media * 26) / 100))])

In [15]:
def fuzzifyErrorDotsPos(erroD, media, maximus):
    return fz.trapezio(erroD,[(media + ((media * 26) / 100)), (media + ((media * 40) / 100)), maximus, maximus])

In [16]:
def fuzzifyErrorDotPos(erroD, media):
    return fz.triangulo(erroD,[(media - (media + ((media * 8) / 100))),(media - (media + ((media * 18) / 100))),(media - (media + ((media * 28) / 100)))])

In [17]:
def fuzzifyErrorDotZ(erroD, media):
    return fz.triangulo(erroD,[(media - ((media * 10) / 100)), media, (media + ((media * 10) / 100))])

In [18]:
def fuzzifyErrorDotNeg(erroD, media):
    return fz.triangulo(erroD,[(media - ((media * 28) / 100)), (media - ((media * 18) / 100)), (media - ((media * 8) / 100))])

In [19]:
def fuzzifyErrorDotsNeg(erroD, media):
    return fz.trapezio(erroD,[0, 0, (media - ((media * 40) / 100)), (media - ((media * 26) / 100))])

In [20]:
def agrega(regras, maximus, sbot, bot, mid, top, stop, media):
    result = [0] * maximus
    for regra in range(len(regras)):
        for i in range(int(maximus)):
            if regras[regra][0]>0 and i < (media - ((media * 26) / 100)):
                result[i] = min(regras[regra][0], bot[i])
            if regras[regra][1]>0 and i > (media - ((media * 28) / 100)) and i < (media - ((media * 8) / 100)):
                result[i] = min(regras[regra][1], mid[i])
            if regras[regra][2]>0 and i > (media - ((media * 10) / 100)) and i < (media + ((media * 10) / 100)):
                result[i] = min(regras[regra][2], top[i])
            if regras[regra][3]>0 and i > (media + ((media * 8) / 100)) and i< (media + ((media * 28) / 100)):
                result[i] = min(regras[regra][3], bot[i])
            if regras[regra][4]>0 and i > (media + ((media * 26) / 100)):
                result[i] = min(regras[regra][4], mid[i])
    return result

In [21]:
def OutputSugDecr(media, maximus):
    return fz.getTrapPlot(0, maximus, [0, 0, (media - ((media * 40) / 100)), (media - ((media * 26) / 100))], "left")

In [22]:
def OutputDecr(media, maximus):
    return fz.getTriPlot(0, maximus, [(media - ((media * 28) / 100)), (media - ((media * 18) / 100)), (media - ((media * 8) / 100))])

In [23]:
def OutputPlat(media, maximus):
    return fz.getTriPlot(0, maximus, [(media - ((media * 10) / 100)), media, (media + ((media * 10) / 100))])

In [24]:
def OutputCres(media, maximus):
    return fz.getTriPlot(0, maximus, [(media + ((media * 8) / 100)), (media + ((media * 18) / 100)), (media + ((media * 28) / 100))])

In [25]:
def OutputSugCres(media, maximus):
    return fz.getTrapPlot(0, maximus, [(media + ((media * 26) / 100)), (media + ((media * 40) / 100)), maximus, maximus], "right")

In [26]:
def verifRegras(erro, erroD, media, maximus):
    regras = [[0] * 25 for i in range(5)]
    
    fuzzifyErrorsNega = fuzzifyErrorsNeg(erro, media)
    fuzzifyErrorNega = fuzzifyErrorNeg(erro, media)
    fuzzifyErrorZe = fuzzifyErrorZ(erro, media)
    fuzzifyErrorPosi= fuzzifyErrorPos(erro, media)
    fuzzifyErrorsPosi= fuzzifyErrorsPos(erro, media, maximus)
    
    fuzzifyErrorDotsNega = fuzzifyErrorDotsNeg(erroD, media)
    fuzzifyErrorDotNega = fuzzifyErrorDotNeg(erroD, media)
    fuzzifyErrorDotZe = fuzzifyErrorDotZ(erroD, media)
    fuzzifyErrorDotPosi = fuzzifyErrorDotPos(erroD, media)
    fuzzifyErrorDotsPosi = fuzzifyErrorDotsPos(erroD, media, maximus)
    
    regras[0][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotsNega)
    regras[0][1] = min(fuzzifyErrorNega,fuzzifyErrorDotsNega)
    regras[0][2] = min(fuzzifyErrorZe,fuzzifyErrorDotsNega)
    regras[0][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotsNega)
    regras[0][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotsNega)
    
    regras[1][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotNega)
    regras[1][1] = min(fuzzifyErrorNega,fuzzifyErrorDotNega)
    regras[1][2] = min(fuzzifyErrorZe,fuzzifyErrorDotNega)
    regras[1][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotNega)
    regras[1][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotNega)
    
    regras[2][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotZe)
    regras[2][1] = min(fuzzifyErrorNega,fuzzifyErrorDotZe)
    regras[2][2] = min(fuzzifyErrorZe,fuzzifyErrorDotZe)
    regras[2][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotZe)
    regras[2][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotZe)
    
    regras[3][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotPosi)
    regras[3][1] = min(fuzzifyErrorNega,fuzzifyErrorDotPosi)
    regras[3][2] = min(fuzzifyErrorZe,fuzzifyErrorDotPosi)
    regras[3][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotPosi)
    regras[3][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotPosi)
    
    regras[4][0] = min(fuzzifyErrorsNega,fuzzifyErrorDotsPosi)
    regras[4][1] = min(fuzzifyErrorNega,fuzzifyErrorDotsPosi)
    regras[4][2] = min(fuzzifyErrorZe,fuzzifyErrorDotsPosi)
    regras[4][3] = min(fuzzifyErrorPosi,fuzzifyErrorDotsPosi)
    regras[4][4] = min(fuzzifyErrorsPosi,fuzzifyErrorDotsPosi)
    
    return regras

In [27]:
media = aver(selected)
diffe = diff(selected)
first = first(selected)
maximus = max(selected['total_cases'])

target = media - ((media * 28) / 100)

In [28]:
currError = target - diffe

In [29]:
error = currError
dervError = first - diffe

In [30]:

Regras = verifRegras(error[0], dervError[0], media[0],round(maximus))
vA = agrega(Regras, int(maximus), OutputSugDecr(int(media[0]),int(maximus)), OutputDecr(int(media[0]),int(maximus)), 
            OutputPlat(int(media[0]),int(maximus)), OutputCres(int(media[0]),int(maximus)), OutputSugCres(int(media[0]),int(maximus)),media[0])
centroide = fz.getCentroid(vA)

print(error)
print(dervError)
print(centroide)
##

cod_location
0      2.802269e+04
1      8.453256e+03
2      3.523538e+04
3      9.979920e+02
4      2.614264e+03
           ...     
207    5.515200e+02
208    2.148036e+07
209    1.457888e+03
210    9.828328e+03
211    5.490568e+03
Name: total_cases, Length: 212, dtype: float64
cod_location
0         38918.0
1         11665.0
2         48846.0
3          1375.0
4          3566.0
          ...    
207         766.0
208    29713363.0
209        2024.0
210       13605.0
211        7619.0
Name: total_cases, Length: 212, dtype: float64
0


In [31]:
## Codigos: 0-Queda Brusca 1-Queda 2-Platô 3-Subida 4-Subida Brusca

0